## Diseño de Vigas tipo T

### Dimensionamiento

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Dimensionamiento
hf = 20 #cm  Altura de la losa
fy = 4200 #kgf/cm2
fc = 280 #kgf/cm2
Es = 2000000 #kg/cm2


if fc <= 280:
    B1 = 0.85
elif fc > 280 and fc <= 560:
    B1 = 0.85 - (0.05*((fc-280)/70))
elif fc > 560:
    B1 = 0.65


#Altura de la Viga

#1: Simply Supported
#2: One end Continuous
#3: Both ends Continuous
#4: Cantilever

TipoViga = 3
l = 716.28 #cm (Longitud de vano)
f1 = 0.4+(fy/7000)

#Altura mínima (Para concreto de eso normal)
if TipoViga == 1:
    h = (l/16)*f1
elif TipoViga == 2:
    h = (l/18.5)*f1
elif TipoViga == 3:
    h = (l/21)*f1
elif TipoViga == 4:
    h = (l/8)*f1

ht = (((h+hf)/ 5) // 1 + 1) * 5 #cm
print(f'La Altura mínima h de la viga sin Losa es: {h/100} m.')
print(f'La Altura h de la viga total es: {ht/100} m.')

La Altura mínima h de la viga sin Losa es: 0.3410857142857143 m.
La Altura h de la viga total es: 0.55 m.


Verificación del bw según resistencia gravitacional requerida

\begin{align*}
&Mn = Rbd^{2} \\
&R = \rho \cdot f_{y}(1 - 0.59\frac{\rho \cdot f_{y}}{f'_{c}})
\end{align*}

<img src='./Images/Vigas3.PNG' height='355px' width='620px'>

In [10]:
rec = 4 #cm
dest = 1 #cm
dvar = 1.6 #cm
d = ht - rec - dest - (dvar/2)

Mu = 16 #Tonf*m
Mu = Mu*100000 #kgf*cm

p = 0.0181
R = p*fy*(1-(0.59*(p*fy/fc)))
bw1 = Mu/(R*(d**2)) #Valor de bw en caso de tratarse de Losa!!

#Para viga aislada
c1 = 50 #cm Lado paralelo de la columna a la viga en cuestión
c2 = 60 #cm Lado perpendicular de la columna a la viga en cuestión

# Para el caso descrito en la figura (Solo vigas aisladas)
bmin = max((0.3*(ht)), 25)
bmax = max(c2, (0.75*c1))
bw = min(max(bw1, bmin), bmax)

if (0.5*bw) <= hf: #Solo para vigas aisladas
    print('Relación cumplida')
else:
    print('Es necesario aumentar el bw o disminuir el hf para cumplir la relación ')
    
print(f'El bmin es: {bmin} cm')
print(f'El bmax es: {bmax} cm')
print(f'El bw es: {bw} cm')


Relación cumplida
El bmin es: 25 cm
El bmax es: 60 cm
El bw es: 25 cm


#### ACI 6.3.2

6.3.2.1 Para vigas en T no pretensadas que soporten losas monolíticas o mixtas, la anchura efectiva del ala bf incluirá la anchura del alma de la viga bw más una anchura efectiva del ala en voladizo de acuerdo con la Tabla 6.3.2.1, donde h es el espesor de la losa y sw es la distancia libre al alma adyacente.

6.3.2.2 Vigas en T aisladas no pretensadas en las que el ala brida se utiliza para proporcionar un área de compresión adicional deberán tener un espesor de ala mayor o igual a 0,5bw y una anchura efectiva de ala menor o igual a 4bw.

<img src='./Images/vigasT1.PNG' height='255px' width='420px'>
<img src='./Images/VigasT2.PNG' height='250px' width='600px'>

In [11]:
sw = 400 #cm
bf =  min((4*bw), (( (bw + (min((8*hf), (sw/2), (l/8)))) / 5) // 1 + 1) * 5) #cm  (Sólo para viga aisladas)
#bf =  (( (bw + (min((8*hf), (sw/2), (l/8)))) / 5) // 1 + 1) * 5 #cm  (Para Losas)
print(f'El ancho bf = {bf} cm')

El ancho bf = 100 cm


### Calculo de As

Determine si la profundidad a de la distribución rectangular de tensiones será mayor para a = t,

\begin{align*}
C = 0.85 f'_{c} b_{f} t \\
Mn = C(d - \frac{t}{2})
\end{align*}

<img src='./Images/vigaT3.PNG'>

Si a < hf   entonces 

\begin{align*}
\rho = \frac{0.85 f'_{c}}{f_{y}}  \left [ 1-\sqrt{1-\frac{4M_{u}}{1.7\phi f'_{c} b d^{2}}} \right ]
\end{align*}

In [12]:
from sympy import symbols, Eq, solve
phi = 0.9
phiMn = Mu/phi #Mn requerido
C = B1*fc*bf*hf
Mn = C*(d - (hf/2))
A2 = (hf*bf) - (bw*hf)
#A1 = bw*a

if Mn > phiMn:
    print('Eje neutro ubicado en el ala, se puede diseñar como viga rectangular')
    p = (0.85*fc/fy)*(1-np.sqrt(1-((4*Mu)/(1.7*phi*fc*bf*(d**2)))))
    As = p*bf*d
    print(f'Entonces el As = {As} cm2')


else:
    print('Eje neutro ubicado en el alma')
    x = symbols('x')
    ecuacion = Eq((B1*fc*(bw*x)*(d-(x/2)) + (0.85*fc*A2*(d-(hf/2)))), phiMn)
    a = solve(ecuacion, x)
    print(f'Valor de a = {a} cm')
    
    C1 = B1*fc*bw*a
    T1 = C1
    As1 = T1/fy

    C2 = B1*fc*(bf-bw)*hf
    T2 = C2
    As2 = T2/fy

    As = As1 + As2

    #Verificar Asmin
    '''El área mínima de refuerzo de tracción requerida en el Código ACI para 
    secciones rectangulares o en T con alas en compresión (ACI 9.6.1.2)
    
    9.6.1.2 As,min será el mayor de (a) y (b), salvo lo dispuesto en lo dispuesto en 9.6.1.3. 
    Para una viga estáticamente determinada con un ala sometida a tracción, el valor de 
    bw será el menor de bf y 2bw. El valor de fy se limitará a un máximo de 550 MPa.
    '''
    Asmin = max((0.8*np.sqrt(fc)*bw*d/fy), (14*bw*d/fy))

    if As < Asmin:
        As = Asmin
    else:
        pass

    print(f'Entonces el As = {As} cm2')



Eje neutro ubicado en el ala, se puede diseñar como viga rectangular
Entonces el As = 8.740262405725247 cm2
